In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import re
from functools import partial
from tqdm.notebook import tqdm

- Training notebook here: https://www.kaggle.com/shivammiglani/spacy-3-0-transformer-custom-ner
- Does not run within the time limit with all sentences.
- Limited the sentences to be processed by spacy using `keywords`

In [ ]:
!pip uninstall spacy -y

In [ ]:
!pip install -U spacy[transformers] --no-index --find-links=file:///kaggle/input/spacytransformers306/spacy

## Predefined function for preprocessing. 

Lowercasing the dataset will make it harder to identify Entities, therefore, keeping case via clean_text2

Also not changing the index (position) of words in a sentence by other preprocessing to search for exact labels

In [ ]:
def get_raw_txt(_id, path="train"):
    _d = json.loads(open(f"{PATH}/{path}/{_id}.json").read())
    return " ".join([i["text"] for i in _d])

def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def clean_text2(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

keywords = set(["data", "study", "survey"])

# Load the custom model and predict.

In [ ]:
import spacy
from thinc.api import set_gpu_allocator, require_gpu
set_gpu_allocator("pytorch")
require_gpu(0)
nlp = spacy.load("../input/spacycustomnerbest/model-best")

# Submission

In [ ]:
# from spacy import displacy
PATH = "../input/coleridgeinitiative-show-us-the-data/"
tqdm_params = dict(bar_format='{desc}{bar} [ {n} / {total} (remaining: {remaining}) ]', colour="darkgreen")

tqdm.pandas(desc="Text download from JSON files status :",**tqdm_params)
submission = pd.read_csv(f"{PATH}sample_submission.csv")
submission['text'] = submission['Id'].progress_apply(partial(get_raw_txt,path="test"))

tqdm.pandas(desc="Text processing status :",**tqdm_params)
for i in tqdm(submission.index, total=submission.shape[0]):
    sentences = submission.loc[i]['text'].split('.')
    predictions = set()
    for sentence in sentences:
        if len(sentence)>20:
            sentence = clean_text2(sentence)
            if any(kw in sentence.lower() for kw in keywords):
                doc = nlp(sentence)
#                 displacy.render(doc, style="ent", jupyter=True)
                if len(doc.ents) > 0:
                    for ent in doc.ents:
                        predictions.add(clean_text(ent.text))
    submission.loc[i,"PredictionString"] = '|'.join(list(predictions))

submission = submission.drop("text",axis=1)
submission.to_csv("submission.csv", index = False)
submission